In [1]:
corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                    [0, 0, 1, 0, 0, 1, 1, 1],
                    [0, 0, 0, 1, 1, 1, 0, 1]])
for i in range(ni):
    for j in range(nj):
        for k in range(nk):
            

SyntaxError: unexpected EOF while parsing (<ipython-input-1-deb1f478f1fa>, line 7)

In [1]:
class TetMesh:
    def __init__(self,origin,nsteps,step_vector):
        self.origin = np.array(origin) 
        self.step_vector = np.array(step_vector)     
        self.nsteps = np.array(nsteps)
        self.nsteps_cells = self.nsteps - 1
        self.n_cell_x = self.nsteps[0] - 1
        self.n_cell_y = self.nsteps[1] - 1
        self.n_cell_z = self.nsteps[2] - 1
        
    def get_tetra(self,pos):
        pos = np.array(pos)
        # find which cell the points are in
        gi, inside = self.position_to_cell_corners(pos)
        # now determine the tetra from the corners
        tetra = np.dstack([gi[:,[0,6,1,4]],
                        gi[:,[2,4,6,7]],
                        gi[:,[0,4,6,2]],
                        gi[:,[3,5,7,2]],
                        gi[:,[0,4,3,2]],
                        gi[:,[3,7,2,4]]])
        # find the x,y,z coordinates of the tetra nodes
        xi,yi,zi = self.global_index_to_node_index(tetra)
        points = self.node_indexes_to_position(xi,yi,zi)
        points = points.swapaxes(0,1)
        c = np.zeros((points.shape[0],points.shape[2],points.shape[3]))
        vap = pos[:,:,None] - points[:, :,0,:]
        vbp = pos[:,:,None] - points[:, :,1, :]
#         # vcp = p - points[:, 2, :]
#         # vdp = p - points[:, 3, :]

        vab = points[:, :, 1, :] - points[:, :, 0, :]
        vac = points[:, :, 2, :] - points[:, :, 0, :]
        vad = points[:, :, 3, :] - points[:, :, 0, :]
        vbc = points[:, :, 2, :] - points[:, :, 1, :]
        vbd = points[:, :, 3, :] - points[:, :, 1, :]
#         vbp * np.cross(vbd, vbc, axisa=1, axisb=1)
        va = np.einsum('ijk, ikj->ik', vbp, np.cross(vbd, vbc, axisa=1, axisb=1)) / 6.
        vb = np.einsum('ijk, ikj->ik', vap, np.cross(vac, vad, axisa=1, axisb=1)) / 6.
        vc = np.einsum('ijk, ikj->ik', vap, np.cross(vad, vab, axisa=1, axisb=1)) / 6.
        vd = np.einsum('ijk, ikj->ik', vap, np.cross(vab, vac, axisa=1, axisb=1)) / 6.
        
        v = np.einsum('ijk, ikj->ik', vab, np.cross(vac, vad, axisa=1, axisb=1)) / 6.

        c[:,0,:] = va / v
        c[:,1,:] = vb / v
        c[:,2,:] = vc / v
        c[:,3,:] = vd / v
        return c
#         va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
#         vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
#         vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
#         vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
#         v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
#         c = np.zeros((4, npts))
#         print(pos.shape)
#         vap = pos[] - points[]
#         print('nodes',points.shape)
#         print(tetra, gi,inside)
    
    def calc_bary_centre(self,points,position):

        npts = len(e)
        vap = p - points[:, 0, :]
        vbp = p - points[:, 1, :]
        # vcp = p - points[:, 2, :]
        # vdp = p - points[:, 3, :]
        vab = points[:, 1, :] - points[:, 0, :]
        vac = points[:, 2, :] - points[:, 0, :]
        vad = points[:, 3, :] - points[:, 0, :]
        vbc = points[:, 2, :] - points[:, 1, :]
        vbd = points[:, 3, :] - points[:, 1, :]
        vbp * np.cross(vbd, vbc, axisa=1, axisb=1)
        va = np.sum(vbp * np.cross(vbd, vbc, axisa=1, axisb=1), axis=1) / 6.
        vb = np.sum(vap * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        vc = np.sum(vap * np.cross(vad, vab, axisa=1, axisb=1), axis=1) / 6.
        vd = np.sum(vap * np.cross(vab, vac, axisa=1, axisb=1), axis=1) / 6.
        v = np.sum(vab * np.cross(vac, vad, axisa=1, axisb=1), axis=1) / 6.
        c = np.zeros((4, npts))
        c[0, :] = va / v
        c[1, :] = vb / v
        c[2, :] = vc / v
        c[3, :] = vd / v
        return c
    
    
    def inside(self, pos):

        # check whether point is inside box
        inside = np.ones(pos.shape[0]).astype(bool)
        for i in range(3):
            inside *= pos[:, i] > self.origin[None, i]
            inside *= pos[:, i] < self.origin[None, i] + \
                      self.step_vector[None, i] * self.nsteps_cells[None, i]
        return inside    
    def global_indicies(self,indexes):
        indexes = np.array(indexes).swapaxes(0, 2)
        return indexes[:, :, 0] + self.nsteps[None, None, 0] * indexes[:, :,
                                                               1] + \
               self.nsteps[None, None, 0] * self.nsteps[
                   None, None, 1] * indexes[:, :, 2]
    def cell_corner_indexes(self, x_cell_index, y_cell_index, z_cell_index):
        """
        Returns the indexes of the corners of a cell given its location xi,
        yi, zi

        Parameters
        ----------
        x_cell_index
        y_cell_index
        z_cell_index

        Returns
        -------

        """
        xcorner = np.array([0, 1, 0, 0, 1, 0, 1, 1])
        ycorner = np.array([0, 0, 1, 0, 0, 1, 1, 1])
        zcorner = np.array([0, 0, 0, 1, 1, 1, 0, 1])
        xcorners = x_cell_index[:, None] + xcorner[None, :]
        ycorners = y_cell_index[:, None] + ycorner[None, :]
        zcorners = z_cell_index[:, None] + zcorner[None, :]
        return xcorners, ycorners, zcorners
    
    def position_to_cell_corners(self,pos):
        inside = self.inside(pos)
        ix, iy, iz = self.position_to_cell_index(pos)
        cornersx, cornersy, cornersz = self.cell_corner_indexes(ix, iy, iz)
        globalidx = self.global_indicies(
            np.dstack([cornersx, cornersy, cornersz]).T)
        return globalidx, inside
    
    def position_to_cell_index(self,pos):
        ix = pos[:, 0] - self.origin[None, 0]
        iy = pos[:, 1] - self.origin[None, 1]
        iz = pos[:, 2] - self.origin[None, 2]
        ix = ix // self.step_vector[None, 0]
        iy = iy // self.step_vector[None, 1]
        iz = iz // self.step_vector[None, 2]
        return ix.astype(int), iy.astype(int), iz.astype(int)
    
    def node_indexes_to_position(self, xindex, yindex, zindex):

        x = self.origin[0] + self.step_vector[0] * xindex
        y = self.origin[1] + self.step_vector[1] * yindex
        z = self.origin[2] + self.step_vector[2] * zindex

        return np.array([x, y, z])
    
    def global_index_to_node_index(self, global_index):
        """
        Convert from global indexes to xi,yi,zi

        Parameters
        ----------
        global_index

        Returns
        -------

        """
        # determine the ijk indices for the global index.
        # remainder when dividing by nx = i
        # remained when dividing modulus of nx by ny is j

        x_index = global_index % self.nsteps[0, None]
        y_index = global_index // self.nsteps[0, None] % \
                  self.nsteps_cells[1, None]
        z_index = global_index // self.nsteps[0, None] // \
                  self.nsteps[1, None]
        return x_index, y_index, z_index

In [4]:
mesh = TetMesh([0,0,0],[10,10,10],[1.1,1.1,1.1])
c = mesh.get_tetra([[.52,.52,.52],[1.15,1.15,1.15] ])

NameError: name 'np' is not defined

In [5]:
mask = np.all(c >= 0, axis=1)
print(c[1],mask[1])

NameError: name 'np' is not defined

In [6]:
corners = np.array([[0, 1, 0, 0, 1, 0, 1, 1],
                    [0, 0, 1, 0, 0, 1, 1, 1],
                    [0, 0, 0, 1, 1, 1, 0, 1]])


def create_mesh(ni,nj,nk):
    def gi(i,j,k):
        return i+j*(ni+i)+k*(ni+1)*(nj+1)
    i = np.arange(0,ni)
    j = np.arange(0,nj)
    k = np.arange(0,nk)
    ii,jj, kk = np.meshgrid(i,j,k,indexing='ij')
    iii = ii[:,:,:,None]+corners[None,None,None,0,:]
    jjj = jj[:,:,:,None]+corners[None,None,None,1,:]
    kkk = kk[:,:,:,None]+corners[None,None,None,2,:]
    corner_gi = gi(iii,jjj,kkk)
    corner_gi_flat = gi(iii,jjj,kkk)
    corner_gi = corner_gi.reshape(ni*nj*nk,8)
    tetra = np.vstack([corner_gi[:,[0,6,1,4]],
                        corner_gi[:,[2,4,6,7]],
                        corner_gi[:,[0,4,6,2]],
                        corner_gi[:,[3,5,7,2]],
                        corner_gi[:,[0,4,3,2]],
                        corner_gi[:,[3,7,2,4]]])
#     faces = np.vstack([corner_gi[:,[0,1,4]],
#                        corner_gi[:,[0,4,6]],
#                        corner_gi[:,[4,6,7]],
#                        corner_gi[:,[0,6,1]],
#                        corner_gi[:,[2,4,7]],
#                        corner_gi[:,[2,7,6]],
#                        corner_gi[:,[3,4,0]],
#                        corner_gi[:,[3,7,5]],
#                        corner_gi[:,[5,7,2]],
#                        corner_gi[:,[0,4,2]],
#                        corner_gi[:,[2,4,6]],
#                        corner_gi[:,[3,2,7]],
#                        corner_gi[:,[1,4,6]],
#                        corner_gi[:,[0,3,2]],
#                        corner_gi[:,[2,3,4]],
#                        corner_gi[:,[0,2,6]],
#                        corner_gi[:,[0,2,4]]
#                             ])
    
#     unique_faces = {}
#     for i, f in enumerate(faces):
#     #     for 
#         unique_faces[tuple(np.sort(f.tolist()))] = i
#     n = 0
#     # convert this back to numpy
#     faces_np = np.zeros(faces.shape).astype(int)
#     for f in unique_faces:
#         faces_np[n,:] = faces[unique_faces[f],:]
#         unique_faces[f] = n
#         n+=1
#     faces_np = faces_np[:n,:]
    
#     faces_tetra = {}
#     for i, t in enumerate(tetra):
#         for idc in [[0,1,2],[1,2,3],[0,2,3],[3,2,1]]:
#             key = tuple(np.sort(t[idc]))

#             if key in faces_tetra:
#                 faces_tetra[key].add(i)
#             else:
#                 faces_tetra[key] = set([i])
                
#     faces_tetra_np = np.zeros((faces.shape[0],2)).astype(int)
#     faces_tetra_np[:,:] = -1
#     n = 0
# #     for face in faces_tetra:
# #         for i in range(len(faces_tetra[face])):
# #             print(faces_tetra[face])
# #             faces_tetra_np[unique_faces[face],i] = list(faces_tetra[face])[i]
# #         n+=1
        
    return tetra

NameError: name 'np' is not defined

In [7]:
create_mesh(100,100,100)

NameError: name 'create_mesh' is not defined

In [8]:
def build_mesh(ni,nj,nk):
    i = np.arange(0,ni)
    j = np.arange(0,nj)
    k = np.arange(0,nk)
    ii,jj, kk = np.meshgrid(i,j,k,indexing='ij')
    iii = ii[:,:,:,None]+corners[None,None,None,0,:]
    jjj = jj[:,:,:,None]+corners[None,None,None,1,:]
    kkk = kk[:,:,:,None]+corners[None,None,None,2,:]
    def gi(i,j,k):
        return i+j*(ni+1)+k*(nj+1)*(ni+1)
    corner_gi = gi(iii,jjj,kkk)
    corner_gi= corner_gi.reshape(ni*nj*nk,8)
    tetra = np.vstack([corner_gi[:,[0,6,1,4]],
                            corner_gi[:,[2,4,6,7]],
                            corner_gi[:,[0,4,6,2]],
                            corner_gi[:,[3,5,7,2]],
                            corner_gi[:,[0,4,3,2]],
                            corner_gi[:,[3,7,2,4]]])
    faces = np.vstack([corner_gi[:,[0,1,4]],
                       corner_gi[:,[0,4,6]],
                       corner_gi[:,[4,6,7]],
                       corner_gi[:,[0,6,1]],
                       corner_gi[:,[2,4,7]],
                       corner_gi[:,[2,7,6]],
                       corner_gi[:,[3,4,0]],
                       corner_gi[:,[3,7,5]],
                       corner_gi[:,[5,7,2]],
                       corner_gi[:,[0,4,2]],
                       corner_gi[:,[2,4,6]],
                       corner_gi[:,[3,2,7]],
                       corner_gi[:,[1,4,6]],
                       corner_gi[:,[0,3,2]],
                       corner_gi[:,[2,3,4]],
                       corner_gi[:,[0,2,6]],
                       corner_gi[:,[0,2,4]]

                            ])

#     # find the unique faces within the list of faces
#     # use a tuple as a hash for a dictionary where the tuple
#     # is a sorted list of the elements in the face and store
#     # the original location in the faces array
#     unique_faces = {}
#     for i, f in enumerate(faces):
#     #     for 
#         unique_faces[tuple(set(f.tolist()))] = i
#     n = 0
#     # convert this back to numpy
#     faces_np = np.zeros(faces.shape).astype(int)
#     for f in unique_faces:
#         faces_np[n,:] = faces[unique_faces[f],:]
#         unique_faces[f] = n
#         n+=1
#     faces_np = faces_np[:n,:]
#     # find which faces belong to which tetra using the same approach as before
#     faces_tetra = {}
#     for i, t in enumerate(tetra):
#         for idc in [[0,1,2],[1,2,3],[0,2,3],[3,2,1]]:
#             key = tuple(set(t[idc]))

#             if key in faces_tetra:
#                 faces_tetra[key].add(i)
#             else:
#                 faces_tetra[key] = set([i])
#     faces_tetra_np = np.zeros((faces.shape[0],2)).astype(int)
#     faces_tetra_np[:,:] = -1
#     n = 0
#     for face in faces_tetra:
#         for i in range(len(faces_tetra[face])):
#             faces_tetra_np[unique_faces[face],i] = list(faces_tetra[face])[i]
#         n+=1

In [9]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.add_collection(Poly3DCollection(nodes[triangle]))

plt.show()

NameError: name 'nodes' is not defined

In [ ]:
%load_ext snakeviz

In [ ]:
%%timeit
build_mesh(100,100,100)


In [ ]:
internal_face_mask = ~np.any(faces_tetra_np[:n,:] == -1, axis=1)

In [10]:
 
def cg(double [:,:,:] EG, long long [:,:] neighbours, long long [:,:] elements,double [:,:] nodes, long long [:] region):
    cdef int Nc, Na, i,Ns, j, ne, ncons, e, n, neigh
    Nc = 5 #numer of constraints shared nodes + independent
    Na = 4 #number of nodes
    Ns = Na -1
    ne = len(neighbours)
    ncons = 0
    cdef int [:] flag = np.zeros(ne,dtype=np.int32)
    cdef double [:,:] c = np.zeros((len(neighbours)*4,Nc))
    cdef long long [:,:] idc = np.zeros((ne*4,5),dtype=np.int64)
    cdef long long [3] common
    cdef double [:] norm = np.zeros((3))
    cdef double [:,:] shared_pts = np.zeros((3,3))
    cdef double [:] v1 = np.zeros(3)
    cdef double [:] v2 = np.zeros(3)
    cdef double [:,:] e1
    cdef double [:,:] e2
    cdef double area = 0
    cdef long long [:] idl  = np.zeros(4,dtype=np.int64)
    cdef long long [:] idr = np.zeros(4,dtype=np.int64)
    for e in range(ne):
        idl = elements[e,:]
        e1 = EG[e,:,:]
        flag[e] = 1
        # if not in region then skip this tetra
        if region[idl[0]] == 0 or region[idl[1]] == 0 or region[idl[2]] == 0 or region[idl[3]] == 0:
            continue
        for n in range(4):

            neigh = neighbours[e,n]
            idr = elements[neigh,:]

            if flag[neigh]== 1:
                continue
            if neigh == -1:
                continue
            # if not in region then skip this tetra
            if region[idr[0]] == 0 or region[idr[1]] == 0 or region[idr[2]] == 0 or region[idr[3]] == 0:
                continue
            e2 = EG[neigh,:,:]


            
            for i in range(Nc):
                idc[ncons,i] = -1

            i = 0
            for itr_right in range(Na):
                for itr_left in range(Na):
                    if idl[itr_left] == idr[itr_right]:
                        common[i] = idl[itr_left]
                        i+=1
            for j in range(3):
                for k in range(3):
                    shared_pts[j][k] = nodes[common[j]][k]#common
            for i in range(3):
                v1[i] = shared_pts[0,i] - shared_pts[1,i]
                v2[i] = shared_pts[2,i]-shared_pts[1,i]
            norm[0] = v2[2]*v1[1] - v1[2]*v2[1]
            norm[1] = v1[2]*v2[0] - v1[0]*v2[2]
            norm[2] = v1[0]*v2[1] - v1[1]*v2[0]

            # we want to weight the cg by the area of the shared face
            # area of triangle is half area of parallelogram
            # https://math.stackexchange.com/questions/128991/how-to-calculate-the-area-of-a-3d-triangle

            area = 0.5*np.linalg.norm(norm)
            for itr_left in range(Na):
                idc[ncons,itr_left] = idl[itr_left]
                for i in range(3):
                    c[ncons,itr_left] += norm[i]*e1[i][itr_left]*area
            next_available_position = Na
            for itr_right in range(Na):
                common_index = -1
                for itr_left in range(Na):
                    if idc[ncons,itr_left] == idr[itr_right]:
                        common_index = itr_left

                position_to_write = 0
                if common_index != -1:
                    position_to_write = common_index
                else:
                    position_to_write = 4#next_available_position
                    next_available_position+=1
                idc[ncons,position_to_write] = idr[itr_right]
                for i in range(3):
                    c[ncons,position_to_write] -= norm[i]*e2[i][itr_right]*area
            ncons+=1
    return idc, c, ncons

SyntaxError: invalid syntax (<ipython-input-10-4fd31a823218>, line 1)

In [11]:
def numpy_cg(faces,faces_tetra,nodes,tetras,grad):
    shared_pts = nodes[faces]#common
    v1 = shared_pts[0,:] - shared_pts[1,:]
    v2 = shared_pts[2,:] - shared_pts[1,:]
    norm = np.cross(v1,v2,axisa=1,axisb=1)
    # we want to weight the cg by the area of the shared face
    # area of triangle is half area of parallelogram
    # https://math.stackexchange.com/questions/128991/how-to-calculate-the-area-of-a-3d-triangle
    area = 0.5*np.linalg.norm(norm)
    e1 = grad[faces_tetra[:,0]]
    e2 = grad[faces_tetra[:,1]]
    
    c 
    for itr_left in range(Na):
        idc[ncons,itr_left] = idl[itr_left]
        for i in range(3):
            c[ncons,itr_left] += norm[i]*e1[i][itr_left]*area
    next_available_position = Na
    for itr_right in range(Na):
        common_index = -1
        for itr_left in range(Na):
            if idc[ncons,itr_left] == idr[itr_right]:
                common_index = itr_left

        position_to_write = 0
        if common_index != -1:
            position_to_write = common_index
        else:
            position_to_write = 4#next_available_position
            next_available_position+=1
        idc[ncons,position_to_write] = idr[itr_right]
        for i in range(3):
            c[ncons,position_to_write] -= norm[i]*e2[i][itr_right]*area

In [12]:
# loop over faces to tetra relations
for face in faces_tetra:
    # loop over tetra global index in faces
    for i in range(len(faces_tetra[face])):
#         print(list(faces_tetra[face])[i],unique_faces[face])
        i_ = 0
        while tetra_faces[list(faces_tetra[face])[i],i_] >  -1:
            i_+=1
        tetra_faces[list(faces_tetra[face])[i],i_] = unique_faces[face]
        


NameError: name 'faces_tetra' is not defined

In [13]:
tetra1 = np.array([0,1,2,3]).astype(int)
tetra2 = np.array([4,1,2,3]).astype(int)
face = np.array([1,2,3]).astype(int)



NameError: name 'np' is not defined

In [14]:
print(tetra_faces.shape)
tetra_faces[np.any(tetra_faces == -1, axis=1)].shape

NameError: name 'tetra_faces' is not defined

In [15]:
plt.triplot(xx.flatten(),yy.flatten(),tri)

NameError: name 'xx' is not defined

In [16]:
def create_surface(bounding_box,nstep):
    x = np.linspace(bounding_box[0,0],bounding_box[1,0],nstep[0])#
    y = np.linspace(bounding_box[0,1],bounding_box[1,1],nstep[1])
    xx, yy = np.meshgrid(x,y,indexing='xy')
    def gi(i,j):
        return i+j*nstep[0]
    corners = np.array([[0,1,0,1],[0,0,1,1]])
    i = np.arange(0, nstep[0]-1)

    j = np.arange(0, nstep[1]-1)
    ii,jj = np.meshgrid(i,j,indexing='ij')
    corner_gi = gi(ii[:,:,None]+corners[None,None,0,:,],jj[:,:,None]+corners[None,None,1,:,])
    corner_gi = corner_gi.reshape((nstep[0]-1)*(nstep[1]-1),4)
    tri = np.vstack([corner_gi[:,:3],corner_gi[:,1:]])
    return tri, xx.flatten(),yy.flatten()

In [17]:
tri, xx, yy = create_surface(np.array([[0,0],[1,1]]),(3,4))
plt.triplot(xx,yy,tri)
print(tri, np.array([xx,yy]).T)

NameError: name 'np' is not defined

In [18]:
def create_surface(bounding_box,nstep):
    x = np.linspace(bounding_box[0,0],bounding_box[1,0],nstep[0])#
    y = np.linspace(bounding_box[0,1],bounding_box[1,1],nstep[1])
    xx, yy = np.meshgrid(x,y,indexing='xy')
    def gi(i,j):
        return i+j*nstep[0]
    corners = np.array([[0,1,0,1],[0,0,1,1]])
    i = np.arange(0, nstep[0]-1)

    j = np.arange(0, nstep[1]-1)
    ii,jj = np.meshgrid(i,j,indexing='ij')
    corner_gi = gi(ii[:,:,None]+corners[None,None,0,:,],jj[:,:,None]+corners[None,None,1,:,])
    corner_gi = corner_gi.reshape((nstep[0]-1)*(nstep[1]-1),4)
    tri = np.vstack([corner_gi[:,:3],corner_gi[:,1:]])
    return tri, xx.flatten(),yy.flatten()
def create_box(bounding_box,nsteps):
    #top and bottom
    nsteps = np.array(nsteps)
    tri, xx, yy = create_surface(bounding_box[0:2,:],nsteps[0:2])
        
    print(xx.shape, yy.shape)


    zz = np.zeros(xx.shape)
    zz[:] = bounding_box[1,2]
    tri = np.vstack([tri,tri+np.max(tri)+1])
    xx = np.hstack([xx,xx])
    yy = np.hstack([yy,yy])
    
    z = np.zeros(zz.shape)
    z[:] = bounding_box[0,2]
    zz = np.hstack([zz,z])
    # y faces
    bb = np.copy(bounding_box[:,[0,2]])
#     bb[1,1] = bounding_box[1,0]
#     bb[0,1] = bounding_box[0,0]
#     bb[1,0] = bounding_box[1,1]
#     bb[0,0] = bounding_box[0,1]
    t, x, z = create_surface(bb,nsteps[[0,2]])
    tri = np.vstack([tri,t+np.max(tri)+1])
    y = np.zeros(x.shape)
    y[:] = boundary_points[0,1]
    xx = np.hstack([xx,x])
    zz = np.hstack([zz,z])
    yy = np.hstack([yy,y])
    
    tri = np.vstack([tri,t+np.max(tri)+1])
    y[:] = boundary_points[1,1]
    xx = np.hstack([xx,x])
    zz = np.hstack([zz,z])
    yy = np.hstack([yy,y])

    # x faces
    t, y, z = create_surface(bounding_box[:,[1,2]],nsteps[[1,2]])
    tri = np.vstack([tri,t+np.max(tri)+1])
    x = np.zeros(y.shape)
    x[:] = boundary_points[0,0]
    xx = np.hstack([xx,x])
    zz = np.hstack([zz,z])
    yy = np.hstack([yy,y])
    
    tri = np.vstack([tri,t+np.max(tri)+1])
    x[:] = boundary_points[1,0]
    xx = np.hstack([xx,x])
    zz = np.hstack([zz,z])
    yy = np.hstack([yy,y])
    
    return tri, xx, yy, zz

In [19]:
origin = (0,0,0)
maximum = (10000,10000,5000)
boundary_points = np.zeros((2,3))
boundary_points[0,0] = origin[0] 
boundary_points[0,1] = origin[1] 
boundary_points[0,2] = origin[2]
boundary_points[1,0] = maximum[0]
boundary_points[1,1] = maximum[1]
boundary_points[1,2] = maximum[2]
tri, xx, yy, zz = create_box(boundary_points,(10,10,5))
n = 0
# for i in range(len(xx)):
#     if (xx[i],yy[i],zz[i]) in points:
#         tri[tri==i] = points[xx[i],yy[i],zz[i]]
#     points[(xx[i],yy[i],zz[i])] = n
#     tri[tri==i] = n
#     n+=1

# points_np = np.zeros((n,3))
# for k in points.keys():
#     points_np[points[k],:] = np.array(k)
# pointsp
# points[i,:]
points = np.zeros((len(xx),3))#
points[:,0] = xx
points[:,1] = yy
points[:,2] = zz#np.array(xx,yy,zz).T


NameError: name 'np' is not defined

In [20]:
import lavavu

In [21]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [22]:
lv = lavavu.Viewer()

surf = lv.triangles('test')
surf.vertices(points)
surf.indices(tri)
# surf['colour'] = 'blue'
surf.values(points[:,0],'num')
surf['colourby'] = 'num'
cmap = 'jet'
surf.colourmap(cmap)
print(points.shape[0])
cpoints = lv.points('points',size=10)
cpoints.vertices(points)
lv.control.Panel()
lv.control.show()

#lv.interactive()

NameError: name 'points' is not defined

In [ ]:
interact(plot_triangles,ntri)

In [ ]:
bounding_box = np.array([[0,0],[3,1]])
tri, x, y = create_surface(bounding_box,(20,10))
z = np.zeros(x.shape)

points = np.zeros((x.shape[0],3))
points[x<=1,0] = x[x<=1]
points[x<=1,1] = y[x<=1]
points[x<=1,2] = z[x<=1]

mask = np.logical_and(x>1, x<2)

points[mask,0] = np.max(x[x<=1])#1#z[x>1]
points[mask,1] = y[mask]
points[mask,2] = x[mask]-np.max(x[x<=1])
mask = x>2
points[x>2,0] = x[x>2]-np.max(points[mask,0])-np.max(x[x<=1])
points[x>2,1] = y[x>2]
points[x>2,2] = z[x>2] + np.max(points[mask,2] )

In [ ]:
boundary_points[0:2,:]

In [ ]:
boundary_points[0:1,:]

In [ ]:
boundary_points[1,[0,2]]